# CV RAG Query Notebook

Tento notebook umožňuje testování dotazů do natrénovaného RAG systému.

## Předpoklady:
- Natrénovaný vectorstore (spusťte nejdřív `training.ipynb` nebo `train.py`)
- Existující ChromaDB databáze v `./chroma_db/`
- Existující docstore v `./chroma_db/docstore/`

## Co tento notebook dělá:
1. Načte existující vectorstore a docstore z disku
2. Inicializuje retriever (bez vytváření nových embeddingů!)
3. Umožní testovat dotazy jako v chatu
4. Zobrazí nalezené kandidáty s kontextem

## Import knihoven

In [ ]:
import logging
from pathlib import Path
import sys

# Přidat parent directory do sys.path pro importy
sys.path.insert(0, str(Path.cwd().parent))

# Nastavení logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import aplikačních modulů
from src.config import AppConfig
from src.embeddings import EmbeddingsManager
from src.vector_store import VectorStoreManager
from src.parent_retriever import CVParentRetriever
from src.rag_chain import CVRAGChain

print("✓ Knihovny načteny")

## Načtení konfigurace

In [ ]:
config = AppConfig()

print("\n📋 Konfigurace:")
print(f"  Vector store: {config.rag.persist_directory}")
print(f"  Collection: {config.rag.collection_name}")
print(f"  Top K results: {config.rag.top_k}")

## Kontrola existence natrénovaných dat

In [ ]:
persist_dir = Path(config.rag.persist_directory)
docstore_dir = persist_dir / "docstore"

print("\n🔍 Kontrola existence dat:")
print("=" * 80)

if not persist_dir.exists():
    print(f"\n❌ ERROR: Vector store neexistuje na {persist_dir}")
    print("   Spusťte nejdřív training.ipynb nebo train.py!")
    raise FileNotFoundError(f"Vector store not found: {persist_dir}")
else:
    print(f"✓ Vector store nalezen: {persist_dir}")

if not docstore_dir.exists():
    print(f"\n❌ ERROR: Docstore neexistuje na {docstore_dir}")
    print("   Spusťte nejdřív training.ipynb nebo train.py!")
    raise FileNotFoundError(f"Docstore not found: {docstore_dir}")
else:
    docstore_files = list(docstore_dir.glob("*"))
    print(f"✓ Docstore nalezen: {len(docstore_files)} parent chunks")

print("\n✅ Všechna data jsou k dispozici")

## Setup Embeddings

In [ ]:
print("\n" + "="*80)
print("Setup Embeddings")
print("="*80)

embeddings_mgr = EmbeddingsManager(config.azure)

print("\n✓ Embeddings manager připraven")

## Načtení Vector Store z disku

In [ ]:
print("\n" + "="*80)
print("Načítání Vector Store")
print("="*80)

vs_manager = VectorStoreManager(
    config.rag,
    embeddings_mgr.get_embeddings()
)

# Načtení existujícího vectorstore
vectorstore = vs_manager.load_vectorstore()

if vectorstore is None:
    raise ValueError("Failed to load vectorstore")

print("\n✓ Vector store načten")

# Statistiky
stats = vs_manager.get_stats()
print(f"\n📊 Statistiky:")
print(f"  Child chunks v databázi: {stats['document_count']}")

## Načtení Parent Document Retriever

**DŮLEŽITÉ:** 
- Používáme `load_from_existing_store()` místo `initialize_retriever()`
- Nevytváříme nové embeddingy!
- Jen načítáme existující data z disku

In [ ]:
print("\n" + "="*80)
print("Načítání Parent Document Retriever")
print("="*80)

retriever = CVParentRetriever(
    config=config.rag,
    vectorstore=vectorstore,
    azure_config=config.azure
)

# Načtení z existujícího store (BEZ vytváření nových embeddingů)
retriever.load_from_existing_store()

print("\n✓ Retriever načten z disku")

# Statistiky
stats = retriever.get_stats()
print(f"\n📊 Statistiky retrieveru:")
print(f"  Parent chunks: {stats['parent_chunks']}")
print(f"  Child chunks: {stats['child_chunks']}")

## Test Simple Retrieval

Vyzkoušejte jednoduché vyhledávání - vrací parent chunks (velké kusy textu).

In [ ]:
print("\n" + "="*80)
print("Test Simple Retrieval")
print("="*80)

query = "Python developer with machine learning experience"

print(f"\n🔍 Dotaz: '{query}'")
print("-" * 80)

results = retriever.retrieve(query, top_k=3)

print(f"\nNalezeno {len(results)} výsledků:\n")

for i, doc in enumerate(results, 1):
    candidate_name = doc.metadata.get("candidate_name", "Unknown")
    content_length = len(doc.page_content)
    
    print(f"{i}. {candidate_name}")
    print(f"   Délka parent chunku: {content_length} znaků")
    print(f"   Metadata: {doc.metadata}")
    print(f"\n   Obsah:")
    print("-" * 80)
    print(doc.page_content[:500])  # První 500 znaků
    print("...\n")

## Setup RAG Chain s LLM

Nyní připravíme kompletní RAG chain, který:
1. Vyhledá relevantní CV
2. Pošle je do LLM jako kontext
3. LLM odpoví na váš dotaz

In [ ]:
print("\n" + "="*80)
print("Setup RAG Chain")
print("="*80)

rag_chain = CVRAGChain(
    config=config.azure,
    retriever=retriever
)

print("\n✓ RAG Chain připraven")
print(f"  LLM Model: {config.azure.llm_deployment}")
print(f"  Temperature: {config.azure.temperature}")

## Test RAG Chain - Dotaz s LLM odpovědí

Toto je jako chat - zadáte otázku a dostanete odpověď od LLM.

In [ ]:
print("\n" + "="*80)
print("Test RAG Chain")
print("="*80)

query = "Who are the best Python developers with AWS experience?"

print(f"\n❓ Otázka: {query}")
print("-" * 80)

response = rag_chain.invoke(query)

print(f"\n💬 Odpověď LLM:")
print("=" * 80)
print(response.answer)
print("=" * 80)

print(f"\n📚 Použité kontexty ({len(response.retrieved_contexts)}):")
for i, ctx in enumerate(response.retrieved_contexts, 1):
    print(f"\n{i}. {ctx.candidate_name}")
    print(f"   Preview: {ctx.content[:200]}...")

## Interaktivní Chat Session

Nyní můžete zadávat vlastní dotazy a dostat odpovědi jako v chatu.

In [ ]:
# Funkce pro hezký výpis
def ask_question(question: str, show_context: bool = True):
    """
    Zeptá se RAG systému a vypíše odpověď.
    
    Args:
        question: Otázka
        show_context: Zda zobrazit použité kontexty
    """
    print("\n" + "="*80)
    print(f"❓ {question}")
    print("="*80)
    
    response = rag_chain.invoke(question)
    
    print(f"\n💬 Odpověď:")
    print("-" * 80)
    print(response.answer)
    print("-" * 80)
    
    if show_context:
        print(f"\n📚 Nalezení kandidáti ({len(response.retrieved_contexts)}):")
        for i, ctx in enumerate(response.retrieved_contexts, 1):
            print(f"  {i}. {ctx.candidate_name}")
    
    return response

print("✓ Funkce ask_question() připravena")

### Příklady dotazů

In [ ]:
# Příklad 1: Hledání podle technologie
ask_question("Who has experience with React and TypeScript?")

In [ ]:
# Příklad 2: Hledání podle role
ask_question("Find me a senior backend developer")

In [ ]:
# Příklad 3: Hledání podle kombinace skillů
ask_question("Who knows both Python and Java and has cloud experience?")

In [ ]:
# Příklad 4: Hledání podle domain knowledge
ask_question("Which candidates have experience in finance or banking?")

### Váš vlastní dotaz

Změňte proměnnou `my_question` a spusťte buňku.

In [ ]:
# Zde napište svůj dotaz
my_question = "Tell me about candidates with DevOps skills"

response = ask_question(my_question, show_context=True)

## Pokročilé: Retrieval se scores

Můžete vidět i similarity scores - jak moc se každý chunk hodí k dotazu.

In [ ]:
query = "Python developer"

print(f"\n🔍 Dotaz: '{query}'")
print("=" * 80)

results_with_scores = retriever.retrieve_with_scores(query, top_k=5)

print(f"\nVýsledky se scores:\n")

for i, result in enumerate(results_with_scores, 1):
    print(f"{i}. {result.candidate_name}")
    print(f"   Score: {result.score:.4f}")
    print(f"   Preview: {result.content[:150]}...")
    print()

## Porovnání: Bez LLM vs. S LLM

Ukázka rozdílu mezi prostým retrievalem a RAG s LLM.

In [ ]:
test_query = "Who can work on a machine learning project?"

print("\n" + "="*80)
print("POROVNÁNÍ: Retrieval vs. RAG")
print("="*80)

# 1. Prostý retrieval (bez LLM)
print(f"\n1️⃣ PROSTÝ RETRIEVAL (bez LLM):")
print("-" * 80)
retrieval_results = retriever.retrieve(test_query, top_k=3)
print(f"Nalezených kandidátů: {len(retrieval_results)}")
for doc in retrieval_results:
    print(f"  - {doc.metadata.get('candidate_name', 'Unknown')}")

# 2. RAG s LLM
print(f"\n2️⃣ RAG S LLM:")
print("-" * 80)
rag_response = rag_chain.invoke(test_query)
print(rag_response.answer)

print("\n" + "="*80)
print("Rozdíl: LLM zpracuje raw data a vytvoří srozumitelnou odpověď")
print("="*80)

## Shrnutí

✅ Query notebook připraven!

**Co jsme udělali:**
1. ✓ Načetli existující vectorstore z disku
2. ✓ Načetli docstore s parent chunks
3. ✓ Inicializovali retriever (BEZ vytváření nových embeddingů)
4. ✓ Otestovali retrieval i RAG chain

**Jak to funguje:**
1. Váš dotaz → vytvoří se embedding
2. Hledání v child chunks (vectorstore) → najdou se relevantní kousky
3. Mapování na parent chunks (docstore) → vrátí se velké kusy kontextu
4. LLM zpracuje kontext → vygeneruje odpověď

**Výhody oproti původnímu řešení:**
- ✅ Parent chunks se načítají z disku (ne fragmentované child chunks)
- ✅ Kompletní kontext pro každého kandidáta
- ✅ Rychlé načítání (žádné nové embeddingy)
- ✅ Persistence mezi restarty